*This analysis is preliminary and based on a toy dataset (n=2), intended solely to validate the inference and analysis pipeline. No performance conclusions are drawn.*

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [38]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   src/inference.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/
	notebooks/inference.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [39]:
!git add data/predictions_with_analysis.csv notebooks/inference.ipynb
!git commit -m "Add notebook for analysis of batch inference results"
!git push

[main a4bebc3] Add notebook for analysis of batch inference results
 2 files changed, 194 insertions(+)
 create mode 100644 data/predictions_with_analysis.csv
 create mode 100644 notebooks/inference.ipynb
Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.64 KiB | 1.64 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: This repository moved. Please use the new location:
remote:   https://github.com/Masesun/nlp-finetuning-bert.git
To https://github.com/masesun/nlp-finetuning-bert.git
   5c10ac7..a4bebc3  main -> main


In [46]:
import pandas as pd

df = pd.read_csv("../predictions.csv")

df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../predictions.csv'

In [47]:
!pwd

/content/nlp-finetuning-bert


In [48]:
!ls /content

drive  nlp-finetuning-bert  sample_data


In [51]:
import os

assert os.path.exists("data/predictions.csv"), "Brak data/predictions.csv w repo"

In [52]:
import pandas as pd

df = pd.read_csv("data/predictions.csv")
df.head()

,text,label,confidence
0,Psilocybin induces rapid structural plasticity...,POSITIVE,0.8810
1,No significant neuroplastic effects were obser...,NEGATIVE,0.5326


In [54]:
!cp "/content/drive/MyDrive/Colab Notebooks/nlp-finetuning-bert.ipynb" \
   "/content/nlp-finetuning-bert/notebooks/error_analysis.ipynb"


In [55]:
!git status
!git add notebooks/error_analysis.ipynb
!git commit -m "Add error analysis notebook"
!git push

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   src/inference.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/inference_samples.csv
	data/predictions.csv
	notebooks/error_analysis.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
[main 6a1cbd0] Add error analysis notebook
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/error_analysis.ipynb
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 2 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 5.94 KiB | 5.94 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
remote: This repository moved. Please use the new location:
r

In [63]:
import os, sys

PROJECT_ROOT = "/content/nlp-finetuning-bert"
os.chdir(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

print("CWD:", os.getcwd())
assert os.path.exists("data/predictions_with_analysis.csv"), "Brak data/predictions_with_analysis.csv"

CWD: /content/nlp-finetuning-bert


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import (
    classification_report,
    confusion_matrix
)

In [65]:
df = pd.read_csv("data/predictions_with_analysis.csv")

df.head(), df.shape

(                                                text     label  confidence  \
 0  Psilocybin induces rapid structural plasticity...  POSITIVE      0.8810   
 1  No significant neuroplastic effects were obser...  NEGATIVE      0.5326   
 
   label_thresholded  
 0          POSITIVE  
 1         UNCERTAIN  ,
 (2, 4))

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        2 non-null      object 
 1   label       2 non-null      object 
 2   confidence  2 non-null      float64
dtypes: float64(1), object(2)
memory usage: 180.0+ bytes


In [61]:
df.isna().sum()

,0
text,0
label,0
confidence,0


In [68]:
df["label"].value_counts()

,count
label,
POSITIVE,1
NEGATIVE,1


In [67]:
df["label_thresholded"].value_counts()

,count
label_thresholded,
POSITIVE,1
UNCERTAIN,1


In [70]:
print(
    classification_report(
        df["label"],
        df["label_thresholded"],
        labels=["NEGATIVE", "POSITIVE"],
        zero_division=0
    )
)


confusion_matrix(df["label"], df["label_thresholded"])

              precision    recall  f1-score   support

    NEGATIVE       0.00      0.00      0.00         1
    POSITIVE       1.00      1.00      1.00         1

   micro avg       1.00      0.50      0.67         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2



array([[0, 0, 1],
       [0, 1, 0],
       [0, 0, 0]])

At the current stage, the model exhibits a tendency to route low-confidence negative samples into the UNCERTAIN category under a conservative threshold (0.7), which is expected behavior given the design of the inference logic.